In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString, Point, MultiPoint
import folium
import dataset

In [ ]:
%%time

db = dataset.connect('sqlite:///stravaweb/strava_activities.sqlite')
BASE_POINT = Point(44.01016, 56.31832)
SIMPLIFY_THRESHOLD = 0.0001
MAX_DISTANCE = 5  # in degrees
crs = {'init': 'epsg:3857'}

df = pd.DataFrame(x for x in db['activities'].all() if x['latlng'])
df['dateteime'] = pd.to_datetime(df['datetime'])
df = df.drop(columns=['time', 'velocity_smooth', 'id'])
gdf = gpd.GeoDataFrame(df.drop(columns=['latlng']), geometry=[LineString([(pt[1], pt[0]) for pt in x]) for x in df.latlng], crs=crs)
gdf['distance_to_base'] = gdf.geometry.distance(BASE_POINT)
gdf['point'] = gpd.GeoSeries([Point((x[0][1], x[0][0])) for x in df.latlng], crs=crs)
print(gdf.shape[0])
gdf.dtypes

In [ ]:
gdf.distance_to_base.hist(bins=10)
gdf.point.plot()

In [ ]:
gdf[gdf.distance_to_base > MAX_DISTANCE].shape[0]

In [ ]:
%%time
from shapely.geometry.collection import GeometryCollection

def highlight_function(feature):
    return {
        'fillColor': 'green',
        'color': 'green',
        'weight': 3,
        'dashArray': '5, 5'
    }


class FeatureCollection(GeometryCollection):
    @property
    def __geo_interface__(self):
        features = []
        for geom in self.geoms:
            features.append({'type': 'Feature', 'geometry': geom.__geo_interface__})
        return dict(type='FeatureCollection', features=features)


my_map = folium.Map(location=[BASE_POINT.y, BASE_POINT.x], zoom_start=12)

for row in gdf[gdf.distance_to_base < MAX_DISTANCE].itertuples():
    tooltip = '{} - {}'.format(row.athlete_name, row.title)
    route = row.geometry.simplify(SIMPLIFY_THRESHOLD, preserve_topology=False)
    geo_obj = folium.GeoJson(
        FeatureCollection([row.point, route]), tooltip=tooltip, 
        highlight_function=highlight_function,
    ).add_to(my_map)
    popup = '<a href="https://www.strava.com/athletes/{athlete_id}" target="_blank">{athlete_name}</a> - <a href="https://www.strava.com/activities/{activity_id}" target="_blank">{activity_name} ({datetime})</a>'.format(
        athlete_id=row.athlete_id, athlete_name=row.athlete_name, activity_id=row.activity_id, activity_name=row.title, datetime=row.datetime
    )
    folium.Popup(popup).add_to(geo_obj)
#     break

my_map

In [ ]:
my_map.save('result.html')